In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import dill as pickle
from scipy.stats import randint, uniform

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer, FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn import metrics
from sklearn.pipeline import make_pipeline

from xgboost import XGBClassifier
from sklearn_pandas import DataFrameMapper

In [2]:
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3952531,287,PAYMENT,27267.81,C1182679756,8958.36,0.00,M1471758739,0.00,0.00,0.0,0.0
3952532,287,PAYMENT,19454.55,C995221159,19425.00,0.00,M79118588,0.00,0.00,0.0,0.0
3952533,287,CASH_OUT,61846.92,C812325508,0.00,0.00,C1299329681,194082.83,255929.76,0.0,0.0
3952534,287,CASH_OUT,39722.69,C767783107,0.00,0.00,C1311396023,366439.25,406161.94,0.0,0.0


In [24]:
def reshape(x):
    return x.reshape(-1, 1)

ct = DataFrameMapper([
    ('type', [FunctionTransformer(reshape, validate=False), LabelBinarizer()]),
    #('nameOrig', [FunctionTransformer(reshape), LabelBinarizer(sparse_output=True)]),
    #('nameDest', [FunctionTransformer(reshape), LabelBinarizer(sparse_output=True)]),
    ('amount', [FunctionTransformer(reshape, validate=False), StandardScaler()]),
    ('oldbalanceOrg', [FunctionTransformer(reshape, validate=False), StandardScaler()]),
    ('newbalanceOrig', [FunctionTransformer(reshape, validate=False), StandardScaler()]),
    ('oldbalanceDest', [FunctionTransformer(reshape, validate=False), StandardScaler()]),
    ('newbalanceDest', [FunctionTransformer(reshape, validate=False), StandardScaler()]),
    ('isFlaggedFraud', None)
])
clf = XGBClassifier()

In [25]:
features = ['type', 'nameOrig', 'nameDest', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFlaggedFraud']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df, df["isFraud"], stratify=df["step"], test_size=0.33, random_state=42)

In [27]:
X_train

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2630704,209,CASH_OUT,195817.56,C340544854,45094.00,0.00,C2068657919,0.00,195817.56,0.0,0.0
2777438,214,PAYMENT,11975.79,C1703209096,18740.00,6764.21,M989381256,0.00,0.00,0.0,0.0
1418695,139,CASH_IN,33584.50,C75153646,1350.00,34934.50,C1086856531,0.00,0.00,0.0,0.0
2869438,227,PAYMENT,14200.19,C1468983453,80998.68,66798.49,M931946816,0.00,0.00,0.0,0.0
1300318,136,CASH_IN,36576.77,C746967996,3121610.52,3158187.29,C1134984520,221785.86,185209.09,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2236515,186,CASH_OUT,330197.02,C1494047281,0.00,0.00,C142493399,970906.42,1301103.44,0.0,0.0
1399004,139,CASH_OUT,250929.63,C156401925,122902.00,0.00,C921382349,255184.98,506114.61,0.0,0.0
1786167,162,CASH_OUT,158128.26,C293375345,0.00,0.00,C189462628,754460.02,912588.28,0.0,0.0
1123751,130,PAYMENT,16672.18,C1200094819,0.00,0.00,M136759652,0.00,0.00,0.0,0.0


In [28]:
X = X_train
y = X["isFraud"].fillna(0)
X = ct.fit_transform(X)

In [29]:
k = pd.DataFrame([df.iloc[1]])
ct.transform(k[features])

array([[ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        -0.38799823, -0.2813039 , -0.28571533, -0.41897863, -0.43406596,
         0.        ]])

In [31]:
clf = XGBClassifier(learning_rate=0.10867386642568048, max_depth=9, n_estimators=122)
clf.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.10867386642568048, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=122, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
pipe = make_pipeline(ct, clf)
pickle.dump(pipe, open("xgb_model_flagged.pkl", 'wb'))

In [33]:
y_pred = pipe.predict(X_test)

In [34]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   1303253
         1.0       1.00      0.60      0.75      1084

    accuracy                           1.00   1304337
   macro avg       1.00      0.80      0.87   1304337
weighted avg       1.00      1.00      1.00   1304337

